## Implementação de novas features usando o dataframe com targets atualizados

In [72]:
import pandas as pd
import numpy as np

In [73]:
data = pd.read_pickle("/data/challenges/hackathon_bigdata_2025/processed_data/merged_transactions_with_targets.pkl")

In [74]:
data

,pdv,zipcode,produto,semana,mes,ano,categoria_pdv,categoria,subcategoria,premise,...,marca,fabricante,descricao,gross_value,net_value,gross_profit,discount,taxes,quantidade,target
0,1001371918471115422,30175.0,1029370090212151375,1,1,2022,Convenience,Package,Lager,Off Premise,...,Michelob Ultra,AB Anheuser Busch Inc,MICHELOB ULTRA 2/12/12 CN,54.299999,49.740486,16.224486,0.000000,4.559516,2.0,Not available
1,1001371918471115422,30175.0,1120490062981954254,1,1,2022,Convenience,Package,Lager,Off Premise,...,Bud Light,AB Anheuser Busch Inc,BUD LIGHT 2/12/8 CN,262.350006,234.992905,62.210903,53.099998,27.357101,18.0,Not available
2,1001371918471115422,30175.0,2239307647969388381,1,1,2022,Convenience,Package,Lager,Off Premise,...,Natural Light,AB Anheuser Busch Inc,NATURAL LIGHT 4/6/16 CN,37.950001,31.870647,8.182648,5.100000,6.079354,2.0,Not available
3,1001371918471115422,30175.0,4353552881410365573,1,1,2022,Convenience,Package,Lager,Off Premise,...,Natural Light,AB Anheuser Busch Inc,NATURAL LIGHT 2/15/12 CN,138.250000,118.302116,33.567116,2.800000,19.947886,7.0,Not available
4,1001371918471115422,30175.0,4797439216678436447,1,1,2022,Convenience,Package,Lager,Off Premise,...,Bud Light Lime,AB Anheuser Busch Inc,BUD LIGHT LIME 2/12/12 CN,26.750000,24.470242,7.524241,0.000000,2.279758,1.0,Not available
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6166028,996297376394222790,30035.0,9024979450937837276,5,12,2022,Convenience,Non-Alcohol,Water,Off Premise,...,Essentia Water,Essentia Water LLC,ESSENTIA WATER 24/20 NR,24.600000,24.600000,8.560000,0.000000,0.000000,1.0,Not available
6166029,997907800111849739,80205.0,3516587220187345473,5,12,2022,Package/Liquor,Package,Lager,Off Premise,...,Natural Ice,AB Anheuser Busch Inc,NATURAL ICE 15/25 CN,220.725006,218.615631,51.665630,24.750000,2.109375,9.0,Not available
6166030,997907800111849739,80205.0,4451855961536069898,5,12,2022,Package/Liquor,Package,Lager,Off Premise,...,Budweiser,AB Anheuser Busch Inc,BUDWEISER 15/25 CN,164.250000,163.078125,46.283127,0.000000,1.171875,5.0,Not available
6166031,997907800111849739,80205.0,7029448080469668619,5,12,2022,Package/Liquor,Package,Lager,Off Premise,...,Bud Light,AB Anheuser Busch Inc,BUD LIGHT 15/25 CN,164.250000,163.078125,46.283127,0.000000,1.171875,5.0,Not available


In [75]:
# Custo da mercadoria = gross_value / quantidade (para o cliente)
# Custo para da mercadoria para o pdv = net_value - gross_profit

# Valor total pago pelo cliente = gross_value; O preço original seria (gross_value + discount)
# Net_value: faturamento da venda descontando impostos
# Gross_value = net_value - custo_de_mercadoria

In [76]:
ids = data['quantidade'] > 0 # update only where quantidade > 0 to avoid division by zero
data['buyer_price_per_unit'] = 0
data.loc[ids, 'buyer_price_per_unit'] = (data.loc[ids, 'gross_value'] / data.loc[ids, 'quantidade']).abs()
data['pdv_cost_per_unit'] = 0
data.loc[ids, 'pdv_cost_per_unit'] = ((data.loc[ids, 'net_value'] - data.loc[ids, 'gross_profit']) / data.loc[ids, 'quantidade']).abs()

/tmp/ipykernel_653421/3990647784.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[27.14999962 14.57500034 18.97500038 ... 32.85       32.85
 37.20000076]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[ids, 'buyer_price_per_unit'] = (data.loc[ids, 'gross_value'] / data.loc[ids, 'quantidade']).abs()
/tmp/ipykernel_653421/3990647784.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[16.75800014  9.59900008 11.84399986 ... 23.35899963 23.35899963
 26.40600014]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[ids, 'pdv_cost_per_unit'] = ((data.loc[ids, 'net_value'] - data.loc[ids, 'gross_profit']) / data.loc[ids, 'quantidade']).abs()


In [77]:
data['pdv_produto'] = data['pdv'].astype(str) + '_' + data['produto'].astype(str)

In [78]:
test_values = np.arange(1, 60, 1)

#for i in test_values:
#    print(f"i: {i}, count: {(data['pdv_produto'].value_counts() >= i).sum()}")


In [ ]:
############ OPÇÃO 1: FILTRAR OS PARES PDV-PRODUTO MAIS VENDIDOS ############

#totais = (
#    data
#    .groupby(["pdv_produto"], as_index=False)["quantidade"]
#    .sum()
#    .rename(columns={"quantidade": "total_vendido"})
#)#

#limite_linhas = 1500000
#n_semanas = 5
#max_pares = limite_linhas // n_semanas   # 300k

#top_pares = (
#    totais
#    .sort_values("total_vendido", ascending=False)
#    .head(max_pares)[["pdv_produto"]]
#)

#filtered = data.merge(top_pares, on=["pdv_produto"], how="inner")
#filtered

In [81]:
###### OPÇÃO 2: ESCOLHER OS PARES PDV-PRODUTO QUE APARECEM EM JANEIRO E QUE TBM SÃO FREQUENTES > 30 SEMANAS, POR EX ####

pdv_produto_jan = data[data.mes == 1]
most_frequent_pdv_produtos = data.pdv_produto.value_counts()

ids = (most_frequent_pdv_produtos >= 30) & (most_frequent_pdv_produtos.index.isin(pdv_produto_jan.pdv_produto.unique()))

# select these IDs:
filtered = data[data.pdv_produto.isin(most_frequent_pdv_produtos[ids].index)]

In [82]:
filtered

,pdv,zipcode,produto,semana,mes,ano,categoria_pdv,categoria,subcategoria,premise,...,gross_value,net_value,gross_profit,discount,taxes,quantidade,target,buyer_price_per_unit,pdv_cost_per_unit,pdv_produto
0,1001371918471115422,30175.0,1029370090212151375,1,1,2022,Convenience,Package,Lager,Off Premise,...,54.299999,49.740486,16.224486,0.00,4.559516,2.0,Not available,27.150000,16.758000,1001371918471115422_1029370090212151375
2,1001371918471115422,30175.0,2239307647969388381,1,1,2022,Convenience,Package,Lager,Off Premise,...,37.950001,31.870647,8.182648,5.10,6.079354,2.0,Not available,18.975000,11.844000,1001371918471115422_2239307647969388381
7,1001371918471115422,30175.0,78720985238870578,1,1,2022,Convenience,Package,Lager,Off Premise,...,37.950001,31.870647,6.642647,5.10,6.079354,2.0,Not available,18.975000,12.614000,1001371918471115422_78720985238870578
8,1001371918471115422,30175.0,8222093130128661995,1,1,2022,Convenience,Package,Other Package,Off Premise,...,42.299999,36.363129,12.061129,0.00,5.936870,2.0,1.0,21.150000,12.151000,1001371918471115422_8222093130128661995
12,1001371918471115422,30175.0,8899583048247637290,1,1,2022,Convenience,Package,Lager,Off Premise,...,79.375000,69.876007,19.056007,11.75,9.498990,5.0,Not available,15.875000,10.164000,1001371918471115422_8899583048247637290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6165970,986798645350155595,30101.0,4623814317972718932,5,12,2022,Convenience,Package,Lager,Off Premise,...,28.400000,26.120241,7.921241,0.00,2.279758,1.0,Not available,28.400000,18.199000,986798645350155595_4623814317972718932
6165973,986798645350155595,30101.0,6529217055583158447,5,12,2022,Convenience,Package,Lager,Off Premise,...,65.309998,56.404694,14.491693,6.15,8.905305,3.0,Not available,21.769999,13.971000,986798645350155595_6529217055583158447
6165978,986798645350155595,30101.0,8222093130128661995,5,12,2022,Convenience,Package,Other Package,Off Premise,...,21.770000,18.801565,4.830560,2.05,2.968435,1.0,Not available,21.770000,13.971005,986798645350155595_8222093130128661995
6166006,990181515273433855,80206.0,8222093130128661995,5,12,2022,Package/Liquor,Package,Other Package,Off Premise,...,46.158001,45.689251,8.001251,14.90,0.468750,2.0,Not available,23.079000,18.844000,990181515273433855_8222093130128661995


In [83]:
#values = (data.pdv_produto.value_counts() >= 40).values
#filtered_data = data[data['pdv_produto'].isin(data.pdv_produto.value_counts()[values].index)]

In [84]:
# if filtered_data.mes != 12, then filter all data with not available target
without_dec = filtered[filtered['mes'] != 12]; with_dec = filtered[filtered['mes'] == 12]
without_dec = without_dec[without_dec.target != 'Not available']

filtered = pd.concat([without_dec, with_dec])

In [85]:
filtered.pdv_produto.nunique()

30020

In [86]:
filtered_train = filtered[filtered.mes.isin([8, 9])]
filtered_val = filtered[filtered.mes.isin([10, 11])]

In [87]:
min_month_train = filtered_train.mes.min()
filtered_train['mes'] = filtered_train.mes - min_month_train
filtered_train['semana_new'] = filtered_train.mes * 5 + filtered_train.semana

min_month_val = filtered_val.mes.min()
filtered_val.mes = filtered_val.mes - min_month_val
filtered_val['semana_new'] = filtered_val.mes * 5 + filtered_val.semana

/tmp/ipykernel_653421/316573106.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_train['mes'] = filtered_train.mes - min_month_train
/tmp/ipykernel_653421/316573106.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_train['semana_new'] = filtered_train.mes * 5 + filtered_train.semana
/tmp/ipykernel_653421/316573106.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [88]:
filtered_train.semana_new.unique(), filtered_val.semana_new.unique()

(array([ 1,  6,  2,  7,  3,  8,  4,  9,  5, 10], dtype=int32),
 array([ 1,  6,  2,  7,  3,  8,  4,  9,  5, 10], dtype=int32))

In [89]:
filtered_train['semana_mes'] =  filtered_train['mes'].astype(str) + "_" + filtered_train['semana'].astype(str)
filtered_val['semana_mes'] =  filtered_val['mes'].astype(str) + "_" + filtered_val['semana'].astype(str)

/tmp/ipykernel_653421/2911584015.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_train['semana_mes'] =  filtered_train['mes'].astype(str) + "_" + filtered_train['semana'].astype(str)
/tmp/ipykernel_653421/2911584015.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_val['semana_mes'] =  filtered_val['mes'].astype(str) + "_" + filtered_val['semana'].astype(str)


In [90]:
#filtered_train[filtered_train.pdv_produto == '1001371918471115422_3407816598812292920'] # Para esse caso, há apenas um target disponível no mês seguinte

In [91]:
filtered_train.pdv_produto.value_counts()

pdv_produto
4944819258719870167_1938760505411922162    10
4944860289835838479_3371701102715312626    10
4944860289835838479_6509066495341489493    10
1333541401405733492_5474460683042756805    10
2430132255826117411_8030392857471983373    10
                                           ..
4864269363226585507_1260174347806822713     1
4688592854110074025_2423393825246291901     1
2790339440802137931_4040509988492387426     1
2737235428009306192_8352471677482341950     1
2737235428009306192_3965996009832298069     1
Name: count, Length: 29381, dtype: int64

In [92]:
filtered_train[filtered_train.pdv_produto == '4440977747777161208_1860061817666925715'].sort_values(by=['mes', 'semana']) # Para esse caso, há targets disponíveis para as 5 semanas do mês seguinte

,pdv,zipcode,produto,semana,mes,ano,categoria_pdv,categoria,subcategoria,premise,...,gross_profit,discount,taxes,quantidade,target,buyer_price_per_unit,pdv_cost_per_unit,pdv_produto,semana_new,semana_mes
782672,4440977747777161208,80550.0,1860061817666925715,1,0,2022,Grocery,Package,Lager,Off Premise,...,27.755005,29.049999,1.26,7.0,21.0,19.504000,15.359000,4440977747777161208_1860061817666925715,1,0_1
2122709,4440977747777161208,80550.0,1860061817666925715,2,0,2022,Grocery,Package,Lager,Off Premise,...,83.265051,87.149998,3.78,21.0,7.0,19.504000,15.358998,4440977747777161208_1860061817666925715,2,0_2
3793880,4440977747777161208,80550.0,1860061817666925715,3,0,2022,Grocery,Package,Lager,Off Premise,...,27.755005,29.049999,1.26,7.0,14.0,19.504000,15.359000,4440977747777161208_1860061817666925715,3,0_3
5202521,4440977747777161208,80550.0,1860061817666925715,4,0,2022,Grocery,Package,Lager,Off Premise,...,27.755005,29.049999,1.26,7.0,6.0,19.504000,15.359000,4440977747777161208_1860061817666925715,4,0_4
5947940,4440977747777161208,80550.0,1860061817666925715,5,0,2022,Grocery,Package,Lager,Off Premise,...,27.755005,29.049999,1.26,7.0,7.0,19.504000,15.359000,4440977747777161208_1860061817666925715,5,0_5
897473,4440977747777161208,80550.0,1860061817666925715,1,1,2022,Grocery,Package,Lager,Off Premise,...,94.730988,63.000000,3.78,21.0,28.0,20.049999,15.359000,4440977747777161208_1860061817666925715,6,1_1
2367682,4440977747777161208,80550.0,1860061817666925715,2,1,2022,Grocery,Package,Lager,Off Premise,...,31.577011,21.000000,1.26,7.0,7.0,20.050001,15.359000,4440977747777161208_1860061817666925715,7,1_2
3909772,4440977747777161208,80550.0,1860061817666925715,3,1,2022,Grocery,Package,Lager,Off Premise,...,72.604004,46.200001,2.52,14.0,7.0,20.725000,15.359000,4440977747777161208_1860061817666925715,8,1_3
5317835,4440977747777161208,80550.0,1860061817666925715,4,1,2022,Grocery,Package,Lager,Off Premise,...,31.116002,19.800000,1.08,6.0,14.0,20.724999,15.359000,4440977747777161208_1860061817666925715,9,1_4
6018354,4440977747777161208,80550.0,1860061817666925715,5,1,2022,Grocery,Package,Lager,Off Premise,...,36.302002,23.100000,1.26,7.0,14.0,20.725000,15.359000,4440977747777161208_1860061817666925715,10,1_5


In [93]:
filtered_train.pdv_produto.value_counts()

pdv_produto
4944819258719870167_1938760505411922162    10
4944860289835838479_3371701102715312626    10
4944860289835838479_6509066495341489493    10
1333541401405733492_5474460683042756805    10
2430132255826117411_8030392857471983373    10
                                           ..
4864269363226585507_1260174347806822713     1
4688592854110074025_2423393825246291901     1
2790339440802137931_4040509988492387426     1
2737235428009306192_8352471677482341950     1
2737235428009306192_3965996009832298069     1
Name: count, Length: 29381, dtype: int64

In [94]:
(filtered_train.pdv_produto.value_counts() == 10).sum()

np.int64(169)

In [95]:
(filtered_train.target == 'Not available').sum()

np.int64(0)

In [96]:
label_week1 = '1_1' # Encontramos as labels para a semana 1 do mês de interesse nas amostras com 1_1
label_week2 = '1_2'
label_week3 = '1_3'
label_week4 = '1_4'
label_week5 = '1_5'
print("Número de labels disponíveis para a semana 1: ", (filtered_train.semana_mes == label_week1).sum())
print("Número de labels disponíveis para a semana 2: ", (filtered_train.semana_mes == label_week2).sum())
print("Número de labels disponíveis para a semana 3: ", (filtered_train.semana_mes == label_week3).sum())
print("Número de labels disponíveis para a semana 4: ", (filtered_train.semana_mes == label_week4).sum())
print("Número de labels disponíveis para a semana 5: ", (filtered_train.semana_mes == label_week5).sum())

Número de labels disponíveis para a semana 1:  15421
Número de labels disponíveis para a semana 2:  15843
Número de labels disponíveis para a semana 3:  15162
Número de labels disponíveis para a semana 4:  16058
Número de labels disponíveis para a semana 5:  1083


In [97]:
def aggr_df(df, categorical_features=[
    'label', 'categoria_pdv', 'categoria'], temporal_features=['quantidade', 'discount', 'gross_value', 'net_value', 'buyer_price_per_unit', 'pdv_cost_per_unit'], week=1, month=1):
    filtered_df = df.copy()

    # Add all features and mantain target column
    first = False
    for feature in temporal_features:
        if feature not in filtered_df.columns:
            raise ValueError(f"Feature {feature} not found in dataframe columns")
        # Aggregate temporal features by pdv_produto and semana_mes using sum
        # Create pivot tables for each temporal feature
        # For example, for quantidade, create features such as "quantidade_08_1", "quantidade_08_2", ..., "quantidade_09_4"
        filtered_X_agg = filtered_df.pivot_table(index='pdv_produto', columns='semana_mes', values=feature, aggfunc='sum')
        filtered_X_agg.columns = [f'{feature}_{col}' for col in filtered_X_agg.columns]
        filtered_X_agg = filtered_X_agg.reset_index()
        if not first:
            first = True
            concat_df = filtered_X_agg.copy()
        else:
            concat_df = concat_df.merge(filtered_X_agg, on='pdv_produto', how='left')

    # For categorical features, create features such as "label", "categoria_pdv", "categoria" using the first value found for that pdv_produto
    categorical_agg = filtered_df.groupby('pdv_produto').agg({
        feature: 'first' for feature in categorical_features
    }).reset_index()
    # Merge all features into a single dataframe
    concat_df =concat_df.merge(categorical_agg, on='pdv_produto', how='left')
    # Get dummies for categorical features
    concat_df = pd.get_dummies(concat_df, columns=categorical_features, drop_first=True)

    # Create 5 target columns for each row; Each target column corresponds to the target for each week (1 to 5), if available, if not fill with None
    for w in range(1, 6):
        label_week = f'{month}_{w}'
        target_agg = filtered_df[filtered_df.semana_mes == label_week][['pdv_produto', 'target']].rename(columns={'target': f'target_week{w}'})
        
        #return target_agg
        concat_df = concat_df.merge(target_agg, on='pdv_produto', how='left')
        # If there are multiple targets for the same pdv_produto, take the first one
        concat_df = concat_df.drop_duplicates(subset=['pdv_produto'], keep='first')
        # If no target is available for that week, fill with None
        if f'target_week{w}' not in concat_df.columns:
            concat_df[f'target_week{w}'] = "Not available"

    # Fill target_df Nan Values with "Not available"
    concat_df[['target_week1', 'target_week2', 'target_week3', 'target_week4', 'target_week5']] = concat_df[
        ['target_week1', 'target_week2', 'target_week3', 'target_week4', 'target_week5']].fillna("Not available")
    concat_df = concat_df.fillna(0)

    return concat_df

In [98]:
#values = (filtered_val.pdv_produto.value_counts() >= 8).values
#filtered_val = filtered_val[filtered_val['pdv_produto'].isin(filtered_val.pdv_produto.value_counts()[values].index)]

In [99]:
features_train = aggr_df(filtered_train)
features_val = aggr_df(filtered_val)

In [101]:
#filtered_train[filtered_train.pdv_produto == '1001371918471115422_3407816598812292920'] # Para esse caso, há apenas um target disponível no mês seguinte

In [102]:
#features_train[features_train.pdv_produto == "1001371918471115422_3407816598812292920"]

In [ ]:
#features_train[features_train.pdv_produto == '4440977747777161208_1860061817666925715']

In [105]:
def combine_cat_cols(train, val):
    for col in val.columns:
        if col not in train.columns:
            train[col] = 0

    for col in train.columns:
        if col not in val.columns:
            val[col] = 0

    return train, val

features_train, features_val = combine_cat_cols(features_train, features_val)

### Agora temos um dataframe de features que organiza todos os targets existentes para um dado pdv_produto.

**Descrição detalhadada:**
1. Ao selecionarmos uma janela de tempo (e.g. 2 meses), extraímos features associadas a cada semana desse período, para que o modelo possa ter uma noção da temporalidade; A feature quantidade 1_5 representa o período mais próximo, enquanto a quantidade0_1, o mais antigo;
2. Cada par pdv_produto presente nesse período pode ou não ter novas instâncias em semanas do mês seguinte; Por isso, organizamos o target dessa forma: target_week1, target_week2...;
3. Essa estrutura permite que treinemos 5 modelos, cada um dedicado a inferir em uma semana do mês futuro aos dados disponibilizados; 
4. Contamos com features de série temporal e features categóricas, que vão ajudar o modelo a aprender características associadas ao pdv e ao produto; Espera-se que o modelo, ao receber em avaliação um novo par pdv-produto, consiga através das features categóricas gerar uma boa predição. Ou seja, o modelo olha para as características da loja e produto para se contextualizar, e então usa features de time series para identificar um padrão dentro desse contexto. (Hipótese)

### Experimento de treinamento:

Em um notebook anterior, experimentamos treinar usando apenas dados de um mês. Nesse, vamos testar uma abordagem que considera mais tempo, e comparar com resultados em ao usar apenas um mês.

**EXP1:**
Treino contém dados dos meses [8, 9]; Validação contém dados do mês [10,11];

**EXP2:**
Treino contém dados dos meses [9]; Validação contém dados do mês [11];

In [106]:
import lightgbm as lgb
import numpy as np

learning_rate = [1e-1, 1e-2, 1e-3]
n_estimators = [200]
max_depth = [10, 30, 50]
num_leaves = [10, 20, 30]
min_child_samples = [5, 10, 20]
reg_alpha = [0.01, 0.1]
reg_lambda = [0.01, 0.1]

def wmape_metric(y_true, y_pred):
    """
    Calcula o Weighted Mean Absolute Percentage Error (wMAPE) para o LightGBM.
    A função precisa retornar uma lista de tuplas no formato:
    (nome_da_metrica, valor_da_metrica, is_higher_better)
    """
    sum_of_true_values = np.sum(np.abs(y_true))
    if sum_of_true_values == 0:
        return [('wmape', 0.0, False)] # Se não há vendas, o erro é zero.

    wmape_score = np.sum(np.abs(y_true - y_pred)) / sum_of_true_values
    
    return [('wmape', wmape_score, False)]

def train_model(X_train, y_train, X_val=None, y_val=None, params = None):
    lgbm = lgb.LGBMRegressor(
        objective='regression',
        metric=None,
        n_estimators=params['n_estimators'],
        learning_rate=params['learning_rate'],
        random_state=41,
        max_depth=params['max_depth'],
        num_leaves=params['num_leaves'],
        min_child_samples=params['min_child_samples'],
        reg_alpha=params['reg_alpha'],
        reg_lambda=params['reg_lambda'],
        n_jobs = 16,
        #num_iterations = 50 if X_val is None else 500,
    )

    lgbm.fit(
        X_train, 
        y_train,
        eval_metric=wmape_metric,
        eval_set=[(X_val, y_val)] if X_val is not None and y_val is not None else None,
        callbacks=[lgb.early_stopping(500, verbose=True)] if X_val is not None and y_val is not None else None,
        #categorical_feature=['label', 'categoria_pdv', 'categoria']
    )
    
    return lgbm

def optimize_hyperparameters(X_train, y_train, X_val, y_val):
    best_wmape = float('inf')
    best_params = None
    best_model = None

    for lr in learning_rate:
        for n_est in n_estimators:
            for md in max_depth:
                for nl in num_leaves:
                    for mcs in min_child_samples:
                        for ra in reg_alpha:
                            for rl in reg_lambda:
                                params = {
                                    'learning_rate': lr,
                                    'n_estimators': n_est,
                                    'max_depth': md,
                                    'num_leaves': nl,
                                    'min_child_samples': mcs,
                                    'reg_alpha': ra,
                                    'reg_lambda': rl
                                }
                                model = train_model(X_train, y_train, X_val, y_val, params)
                                y_pred = model.predict(X_val)
                                wmape = np.sum(np.abs(y_val - y_pred)) / np.sum(np.abs(y_val))

                                if wmape < best_wmape:
                                    best_wmape = wmape
                                    best_params = params
                                    best_model = model
                                    print(f"New best wMAPE: {best_wmape:.4f} with params: {best_params}")

    return best_model, best_params, best_wmape

def calculate_results_for_each_model(train, val, group_size=5):
    best_models = []
    best_params_list = []
    best_wmapes = []

    target_cols = [f'target_week{i+1}' for i in range(5)]
    #X_train = train.drop(columns=target_cols + ['pdv_produto'])
    #X_val = val.drop(columns=target_cols + ['pdv_produto'])


    for week in range(group_size):
        print(f"Optimizing hyperparameters for week {week + 1}")
        train_week = train[train[target_cols[week]] != "Not available"]
        val_week = val[val[target_cols[week]] != "Not available"]

        #return train_week
        X_train_week = train_week.drop(columns=target_cols + ['pdv_produto'])
        y_train_week = train_week[target_cols[week]].astype(float)
        X_val_week = val_week.drop(columns=target_cols + ['pdv_produto'])
        y_val_week = val_week[target_cols[week]].astype(float)

        
        best_model, best_params, best_wmape = optimize_hyperparameters(X_train_week, y_train_week, X_val_week, y_val_week)
        
        best_models.append(best_model)
        best_params_list.append(best_params)
        best_wmapes.append(best_wmape)

    return best_models, best_params_list, best_wmapes

In [107]:
#train_week

In [108]:
best_models, best_params_list, best_wmapes = calculate_results_for_each_model(features_train, features_val, group_size=1)

Optimizing hyperparameters for week 1
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001838 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12504
[LightGBM] [Info] Number of data points in the train set: 15421, number of used features: 92
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 4.732515
Training until validation scores don't improve for 500 rounds
Did not meet early stopping. Best iteration is:
[165]	valid_0's l2: 16.9503	valid_0's wmape: 0.394676
New best wMAPE: 0.3947 with params: {'learning_rate': 0.1, 'n_estimators': 200, 'max_depth': 10, 'num_leaves': 10, 'min_child_samples': 5, 'reg_alpha': 0.01, 'reg_lambda': 0.01}
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-w

In [110]:
best_wmapes # Melhor resultado para a semana 1

[np.float64(0.39405738865526996)]

In [111]:
best_params_list[0]

{'learning_rate': 0.1,
 'n_estimators': 200,
 'max_depth': 30,
 'num_leaves': 30,
 'min_child_samples': 5,
 'reg_alpha': 0.01,
 'reg_lambda': 0.1}

In [112]:
params =  {'learning_rate': 0.1,
 'n_estimators': 200,
 'max_depth': 30,
 'num_leaves': 20,
 'min_child_samples': 5,
 'reg_alpha': 0.1,
 'reg_lambda': 0.1}

In [113]:
models = []
wmapes = []
wmapes_round = []

target_cols = [f'target_week{i+1}' for i in range(5)]
train = features_train.copy()
val = features_val.copy()

for week in range(5):
    print(f"Training model for week {week + 1}")

    train_week = train[train[target_cols[week]] != "Not available"]
    val_week = val[val[target_cols[week]] != "Not available"]

    X_train_week = train_week.drop(columns=target_cols + ['pdv_produto'])
    y_train_week = train_week[target_cols[week]].astype(float)
    X_val_week = val_week.drop(columns=target_cols + ['pdv_produto'])
    y_val_week = val_week[target_cols[week]].astype(float)
    
    model = train_model(X_train_week, y_train_week, X_val_week, y_val_week, params)

    models.append(model)
    wmapes.append(np.sum(np.abs(y_val_week - model.predict(X_val_week))) / np.sum(np.abs(y_val_week)))
    wmapes_round.append(np.sum(np.abs(y_val_week - np.round(model.predict(X_val_week))) / np.sum(np.abs(y_val_week))))

Training model for week 1
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001888 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12504
[LightGBM] [Info] Number of data points in the train set: 15421, number of used features: 92
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 4.732515
Training until validation scores don't improve for 500 rounds
Did not meet early stopping. Best iteration is:
[55]	valid_0's l2: 17.2789	valid_0's wmape: 0.396672
Training model for week 2
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002142 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12835
[Ligh

In [114]:
np.mean(wmapes), np.std(wmapes)

(np.float64(0.4224608492675765), np.float64(0.030322723936251485))

In [115]:
np.mean(wmapes_round), np.std(wmapes_round)

(np.float64(0.42047213698737584), np.float64(0.029575568729881672))

Resultados EXP1: 
WMAPE = 46.23% +- 3.70%

Resultados EXP1 (v2): 
WMAPE = 42.04% +- 2.96%


In [116]:
filtered_train = filtered[filtered.mes.isin([9])]
filtered_val = filtered[filtered.mes.isin([11])]

In [117]:
filtered_train.shape, filtered_val.shape

((63567, 25), (59006, 25))

In [118]:
#min_month_train = filtered_train.mes.min()
filtered_train.loc[:,'mes'] = 0
#filtered_train.loc[:,'semana_new'] = filtered_train.mes * 5 + filtered_train.semana

#min_month_val = filtered_val.mes.min()
filtered_val.loc[:,'mes'] = 0
#filtered_val.loc[:,'semana_new'] = filtered_val.mes * 5 + filtered_val.semana

In [119]:
filtered_train['semana_mes'] =  filtered_train['mes'].astype(str) + "_" + filtered_train['semana'].astype(str)
filtered_val['semana_mes'] =  filtered_val['mes'].astype(str) + "_" + filtered_val['semana'].astype(str)

/tmp/ipykernel_653421/2911584015.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_train['semana_mes'] =  filtered_train['mes'].astype(str) + "_" + filtered_train['semana'].astype(str)
/tmp/ipykernel_653421/2911584015.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_val['semana_mes'] =  filtered_val['mes'].astype(str) + "_" + filtered_val['semana'].astype(str)


In [120]:
label_week1 = '0_1' # Encontramos as labels para a semana 1 do mês de interesse nas amostras com 1_1
label_week2 = '0_2'
label_week3 = '0_3'
label_week4 = '0_4'
label_week5 = '0_5'
print("Número de labels disponíveis para a semana 1: ", (filtered_train.semana_mes == label_week1).sum())
print("Número de labels disponíveis para a semana 2: ", (filtered_train.semana_mes == label_week2).sum())
print("Número de labels disponíveis para a semana 3: ", (filtered_train.semana_mes == label_week3).sum())
print("Número de labels disponíveis para a semana 4: ", (filtered_train.semana_mes == label_week4).sum())
print("Número de labels disponíveis para a semana 5: ", (filtered_train.semana_mes == label_week5).sum())

Número de labels disponíveis para a semana 1:  15421
Número de labels disponíveis para a semana 2:  15843
Número de labels disponíveis para a semana 3:  15162
Número de labels disponíveis para a semana 4:  16058
Número de labels disponíveis para a semana 5:  1083


In [121]:
features_train = aggr_df(filtered_train, month=0)
features_val = aggr_df(filtered_val, month=0)

In [122]:
features_train, features_val = combine_cat_cols(features_train, features_val)

In [123]:
features_train.target_week1.value_counts();

In [124]:
best_models, best_params_list, best_wmapes = calculate_results_for_each_model(features_train, features_val, group_size=1)

Optimizing hyperparameters for week 1
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000933 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5892
[LightGBM] [Info] Number of data points in the train set: 15421, number of used features: 62
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 4.732515
Training until validation scores don't improve for 500 rounds
Did not meet early stopping. Best iteration is:
[195]	valid_0's l2: 18.86	valid_0's wmape: 0.419724
New best wMAPE: 0.4197 with params: {'learning_rate': 0.1, 'n_estimators': 200, 'max_depth': 10, 'num_leaves': 10, 'min_child_samples': 5, 'reg_alpha': 0.01, 'reg_lambda': 0.01}
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise

In [126]:
best_params_list[0]

{'learning_rate': 0.1,
 'n_estimators': 200,
 'max_depth': 30,
 'num_leaves': 30,
 'min_child_samples': 10,
 'reg_alpha': 0.1,
 'reg_lambda': 0.1}

In [127]:
params = {'learning_rate': 0.1,
 'n_estimators': 200,
 'max_depth': 30,
 'num_leaves': 30,
 'min_child_samples': 10,
 'reg_alpha': 0.1,
 'reg_lambda': 0.1}

In [128]:
models = []
wmapes = []
wmapes_round = []

target_cols = [f'target_week{i+1}' for i in range(5)]
train = features_train.copy()
val = features_val.copy()

for week in range(5):
    print(f"Training model for week {week + 1}")

    train_week = train[train[target_cols[week]] != "Not available"]
    val_week = val[val[target_cols[week]] != "Not available"]

    X_train_week = train_week.drop(columns=target_cols + ['pdv_produto'])
    y_train_week = train_week[target_cols[week]].astype(float)
    X_val_week = val_week.drop(columns=target_cols + ['pdv_produto'])
    y_val_week = val_week[target_cols[week]].astype(float)
    
    model = train_model(X_train_week, y_train_week, X_val_week, y_val_week, params)

    models.append(model)
    wmapes.append(np.sum(np.abs(y_val_week - model.predict(X_val_week))) / np.sum(np.abs(y_val_week)))
    wmapes_round.append(np.sum(np.abs(y_val_week - np.round(model.predict(X_val_week))) / np.sum(np.abs(y_val_week))))


Training model for week 1
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000901 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5884
[LightGBM] [Info] Number of data points in the train set: 15421, number of used features: 58
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 4.732515
Training until validation scores don't improve for 500 rounds
Did not meet early stopping. Best iteration is:
[56]	valid_0's l2: 17.833	valid_0's wmape: 0.416065
Training model for week 2
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001274 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6183
[LightGB

In [129]:
np.mean(wmapes), np.std(wmapes)

(np.float64(0.447115906798268), np.float64(0.0461002913435405))

In [130]:
np.mean(wmapes_round), np.std(wmapes_round)

(np.float64(0.44373213324655103), np.float64(0.0431473118005851))

# EXP3 - 3 meses

In [131]:
filtered_train = filtered[filtered.mes.isin([7,8,9])]
filtered_val = filtered[filtered.mes.isin([9,10,11])]

In [132]:
min_month_train = filtered_train.mes.min()
filtered_train.loc[:,'mes'] = filtered_train.mes - min_month_train

min_month_val = filtered_val.mes.min()
filtered_val.loc[:,'mes'] = filtered_val.mes - min_month_val

In [133]:
filtered_train['semana_mes'] =  filtered_train['mes'].astype(str) + "_" + filtered_train['semana'].astype(str)
filtered_val['semana_mes'] =  filtered_val['mes'].astype(str) + "_" + filtered_val['semana'].astype(str)

/tmp/ipykernel_653421/2911584015.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_train['semana_mes'] =  filtered_train['mes'].astype(str) + "_" + filtered_train['semana'].astype(str)
/tmp/ipykernel_653421/2911584015.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_val['semana_mes'] =  filtered_val['mes'].astype(str) + "_" + filtered_val['semana'].astype(str)


In [134]:
label_week1 = '2_1' # Encontramos as labels para a semana 1 do mês de interesse nas amostras com 1_1
label_week2 = '2_2'
label_week3 = '2_3'
label_week4 = '2_4'
label_week5 = '2_5'
print("Número de labels disponíveis para a semana 1: ", (filtered_train.semana_mes == label_week1).sum())
print("Número de labels disponíveis para a semana 2: ", (filtered_train.semana_mes == label_week2).sum())
print("Número de labels disponíveis para a semana 3: ", (filtered_train.semana_mes == label_week3).sum())
print("Número de labels disponíveis para a semana 4: ", (filtered_train.semana_mes == label_week4).sum())
print("Número de labels disponíveis para a semana 5: ", (filtered_train.semana_mes == label_week5).sum())

Número de labels disponíveis para a semana 1:  15421
Número de labels disponíveis para a semana 2:  15843
Número de labels disponíveis para a semana 3:  15162
Número de labels disponíveis para a semana 4:  16058
Número de labels disponíveis para a semana 5:  1083


In [135]:
features_train = aggr_df(filtered_train, month=2)
features_val = aggr_df(filtered_val, month=2)

In [136]:
features_train, features_val = combine_cat_cols(features_train, features_val)

In [202]:
best_models, best_params_list, best_wmapes = calculate_results_for_each_model(features_train, features_val, group_size=1)

Optimizing hyperparameters for week 1
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005132 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 19546
[LightGBM] [Info] Number of data points in the train set: 39837, number of used features: 134
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 3.818548
Training until validation scores don't improve for 500 rounds
Did not meet early stopping. Best iteration is:
[62]	valid_0's l2: 69.0324	valid_0's wmape: 0.434832
New best wMAPE: 0.4348 with params: {'learning_rate': 0.1, 'n_estimators': 200, 'max_depth': 10, 'num_leaves': 10, 'min_child_samples': 5, 'reg_alpha': 0.01, 'reg_lambda': 0.01}
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-w

In [203]:
best_params_list[0]

{'learning_rate': 0.1,
 'n_estimators': 200,
 'max_depth': 30,
 'num_leaves': 30,
 'min_child_samples': 20,
 'reg_alpha': 0.01,
 'reg_lambda': 0.01}

In [137]:
params = {'learning_rate': 0.1,
 'n_estimators': 200,
 'max_depth': 30,
 'num_leaves': 30,
 'min_child_samples': 20,
 'reg_alpha': 0.01,
 'reg_lambda': 0.01}

In [138]:
models = []
wmapes = []
wmapes_round = []

target_cols = [f'target_week{i+1}' for i in range(5)]
train = features_train.copy()
val = features_val.copy()

for week in range(5):
    print(f"Training model for week {week + 1}")

    train_week = train[train[target_cols[week]] != "Not available"]
    val_week = val[val[target_cols[week]] != "Not available"]

    X_train_week = train_week.drop(columns=target_cols + ['pdv_produto'])
    y_train_week = train_week[target_cols[week]].astype(float)
    X_val_week = val_week.drop(columns=target_cols + ['pdv_produto'])
    y_val_week = val_week[target_cols[week]].astype(float)
    
    model = train_model(X_train_week, y_train_week, X_val_week, y_val_week, params)

    models.append(model)
    wmapes.append(np.sum(np.abs(y_val_week - model.predict(X_val_week))) / np.sum(np.abs(y_val_week)))
    wmapes_round.append(np.sum(np.abs(y_val_week - np.round(model.predict(X_val_week))) / np.sum(np.abs(y_val_week))))


Training model for week 1
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002815 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 19053
[LightGBM] [Info] Number of data points in the train set: 15421, number of used features: 109
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 4.732515
Training until validation scores don't improve for 500 rounds
Did not meet early stopping. Best iteration is:
[46]	valid_0's l2: 16.1876	valid_0's wmape: 0.391287
Training model for week 2
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002626 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 19393
[Lig

In [139]:
np.mean(wmapes), np.std(wmapes)

(np.float64(0.4240169808370193), np.float64(0.04057239847466978))

In [140]:
np.mean(wmapes_round), np.std(wmapes_round)

(np.float64(0.4218512711864027), np.float64(0.037339671965118844))

# EXP4 - 4 meses

In [63]:
filtered_train = filtered[filtered.mes.isin([6,7,8,9])]
filtered_val = filtered[filtered.mes.isin([8,9,10,11])]

In [64]:
min_month_train = filtered_train.mes.min()
filtered_train.loc[:,'mes'] = filtered_train.mes - min_month_train

min_month_val = filtered_val.mes.min()
filtered_val.loc[:,'mes'] = filtered_val.mes - min_month_val

In [65]:
filtered_train['semana_mes'] =  filtered_train['mes'].astype(str) + "_" + filtered_train['semana'].astype(str)
filtered_val['semana_mes'] =  filtered_val['mes'].astype(str) + "_" + filtered_val['semana'].astype(str)

/tmp/ipykernel_586632/2911584015.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_train['semana_mes'] =  filtered_train['mes'].astype(str) + "_" + filtered_train['semana'].astype(str)
/tmp/ipykernel_586632/2911584015.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_val['semana_mes'] =  filtered_val['mes'].astype(str) + "_" + filtered_val['semana'].astype(str)


In [66]:
label_week1 = '3_1' # Encontramos as labels para a semana 1 do mês de interesse nas amostras com 1_1
label_week2 = '3_2'
label_week3 = '3_3'
label_week4 = '3_4'
label_week5 = '3_5'
print("Número de labels disponíveis para a semana 1: ", (filtered_train.semana_mes == label_week1).sum())
print("Número de labels disponíveis para a semana 2: ", (filtered_train.semana_mes == label_week2).sum())
print("Número de labels disponíveis para a semana 3: ", (filtered_train.semana_mes == label_week3).sum())
print("Número de labels disponíveis para a semana 4: ", (filtered_train.semana_mes == label_week4).sum())
print("Número de labels disponíveis para a semana 5: ", (filtered_train.semana_mes == label_week5).sum())

Número de labels disponíveis para a semana 1:  39837
Número de labels disponíveis para a semana 2:  53189
Número de labels disponíveis para a semana 3:  40102
Número de labels disponíveis para a semana 4:  41297
Número de labels disponíveis para a semana 5:  1861


In [67]:
features_train = aggr_df(filtered_train, month=3)
features_val = aggr_df(filtered_val, month=3)

In [68]:
features_train, features_val = combine_cat_cols(features_train, features_val)

In [213]:
best_models, best_params_list, best_wmapes = calculate_results_for_each_model(features_train, features_val, group_size=1)

Optimizing hyperparameters for week 1
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006238 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26201
[LightGBM] [Info] Number of data points in the train set: 39837, number of used features: 164
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 3.818548
Training until validation scores don't improve for 500 rounds
Did not meet early stopping. Best iteration is:
[55]	valid_0's l2: 68.4114	valid_0's wmape: 0.441474
New best wMAPE: 0.4415 with params: {'learning_rate': 0.1, 'n_estimators': 200, 'max_depth': 10, 'num_leaves': 10, 'min_child_samples': 5, 'reg_alpha': 0.01, 'reg_lambda': 0.01}
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-w

In [216]:
best_params_list[0]

{'learning_rate': 0.1,
 'n_estimators': 200,
 'max_depth': 30,
 'num_leaves': 30,
 'min_child_samples': 20,
 'reg_alpha': 0.1,
 'reg_lambda': 0.01}

In [69]:
params = {'learning_rate': 0.1,
 'n_estimators': 200,
 'max_depth': 30,
 'num_leaves': 30,
 'min_child_samples': 20,
 'reg_alpha': 0.1,
 'reg_lambda': 0.01}

In [70]:
models = []
wmapes = []
wmapes_round = []

target_cols = [f'target_week{i+1}' for i in range(5)]
train = features_train.copy()
val = features_val.copy()

for week in range(5):
    print(f"Training model for week {week + 1}")

    train_week = train[train[target_cols[week]] != "Not available"]
    val_week = val[val[target_cols[week]] != "Not available"]

    X_train_week = train_week.drop(columns=target_cols + ['pdv_produto'])
    y_train_week = train_week[target_cols[week]].astype(float)
    X_val_week = val_week.drop(columns=target_cols + ['pdv_produto'])
    y_val_week = val_week[target_cols[week]].astype(float)
    
    model = train_model(X_train_week, y_train_week, X_val_week, y_val_week, params)

    models.append(model)
    wmapes.append(np.sum(np.abs(y_val_week - model.predict(X_val_week))) / np.sum(np.abs(y_val_week)))
    wmapes_round.append(np.sum(np.abs(y_val_week - np.round(model.predict(X_val_week))) / np.sum(np.abs(y_val_week))))


Training model for week 1
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006793 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26181
[LightGBM] [Info] Number of data points in the train set: 39837, number of used features: 154
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 3.818548
Training until validation scores don't improve for 500 rounds
Did not meet early stopping. Best iteration is:
[51]	valid_0's l2: 71.0005	valid_0's wmape: 0.433397
Training model for week 2
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006596 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can se

In [71]:
np.mean(wmapes), np.std(wmapes)

(np.float64(0.47502699900639306), np.float64(0.04855968305791016))

In [72]:
np.mean(wmapes_round), np.std(wmapes_round)

(np.float64(0.4669428822135238), np.float64(0.05086563001037789))

**Resultados EXP1 (2 meses):**
WMAPE = 46.23% +- 3.70%

**Resultados EXP2 (1 mes):**
WMAPE = 47.73 +- 4.05%

**Resultados EXP3 (3 meses):**
WMAPE = 45.66 +- 5.53%

**Resultados EXP4 (4 meses):**
WMAPE = 47.5% +- 4.86%

# Com o melhor resultado (EXP1), vamos fazer um treinamento com os últimos dois meses disponíveis:

In [141]:
filtered_train = filtered[filtered.mes.isin([10,11])]
test = filtered[filtered.mes.isin([11,12])]

In [142]:
#values = (test.pdv_produto.value_counts() >= 2).values
#test = test[test['pdv_produto'].isin(test.pdv_produto.value_counts()[values].index)]

In [163]:
test.pdv_produto.nunique()

28909

In [144]:
min_month_train = filtered_train.mes.min()
filtered_train['mes'] = filtered_train.mes - min_month_train

min_month_test = test.mes.min()
test['mes'] = test.mes - min_month_test

/tmp/ipykernel_653421/3253973998.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_train['mes'] = filtered_train.mes - min_month_train
/tmp/ipykernel_653421/3253973998.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['mes'] = test.mes - min_month_test


In [145]:
filtered_train['semana_mes'] =  filtered_train['mes'].astype(str) + "_" + filtered_train['semana'].astype(str)
test['semana_mes'] =  test['mes'].astype(str) + "_" + test['semana'].astype(str)

/tmp/ipykernel_653421/3447113126.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_train['semana_mes'] =  filtered_train['mes'].astype(str) + "_" + filtered_train['semana'].astype(str)
/tmp/ipykernel_653421/3447113126.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['semana_mes'] =  test['mes'].astype(str) + "_" + test['semana'].astype(str)


In [146]:
label_week1 = '1_1' # Encontramos as labels para a semana 1 do mês de interesse nas amostras com 1_1
label_week2 = '1_2'
label_week3 = '1_3'
label_week4 = '1_4'
label_week5 = '1_5'
print("Número de labels disponíveis para a semana 1: ", (filtered_train.semana_mes == label_week1).sum())
print("Número de labels disponíveis para a semana 2: ", (filtered_train.semana_mes == label_week2).sum())
print("Número de labels disponíveis para a semana 3: ", (filtered_train.semana_mes == label_week3).sum())
print("Número de labels disponíveis para a semana 4: ", (filtered_train.semana_mes == label_week4).sum())
print("Número de labels disponíveis para a semana 5: ", (filtered_train.semana_mes == label_week5).sum())

Número de labels disponíveis para a semana 1:  15154
Número de labels disponíveis para a semana 2:  15882
Número de labels disponíveis para a semana 3:  16354
Número de labels disponíveis para a semana 4:  9814
Número de labels disponíveis para a semana 5:  1802


In [147]:
features_train = aggr_df(filtered_train)
features_test = aggr_df(test)

In [148]:
features_test.shape

(28909, 110)

In [149]:
features_train, features_test = combine_cat_cols(features_train, features_test)

In [150]:
models_2nd_deploy = []
wmapes_train_2nd_deploy = []
#predictions_2nd_deploy = []

target_cols = [f'target_week{i+1}' for i in range(5)]
train = features_train.copy()

for week in range(5):
    print(f"Training model for week {week + 1}")

    train_week = train[train[target_cols[week]] != "Not available"]
    val_week = val[val[target_cols[week]] != "Not available"]

    X_train_week = train_week.drop(columns=target_cols + ['pdv_produto'])
    y_train_week = train_week[target_cols[week]].astype(float)
    
    model = train_model(X_train_week, y_train_week, params= params)

    models_2nd_deploy.append(model)
    preds = model.predict(X_train_week)
    #predictions_2nd_deploy.append(preds)
    wmapes_train_2nd_deploy.append(np.sum(np.abs(y_train_week - np.round(model.predict(X_train_week)))) / np.sum(np.abs(y_train_week)))


Training model for week 1
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001429 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12365
[LightGBM] [Info] Number of data points in the train set: 15154, number of used features: 79
[LightGBM] [Info] Start training from score 4.640066
Training model for week 2
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002065 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12519
[LightGBM] [Info] Number of data points in the train set: 15882, number of used features: 79
[LightGBM] [Info] Start training from score 4.644489
Training model for week 3
[LightGBM] [Warning] Found whitespace in feature_names, replace with underli

In [151]:
models

[LGBMRegressor(max_depth=30, metric=None, n_estimators=200, n_jobs=16,
               num_leaves=30, objective='regression', random_state=41,
               reg_alpha=0.01, reg_lambda=0.01),
 LGBMRegressor(max_depth=30, metric=None, n_estimators=200, n_jobs=16,
               num_leaves=30, objective='regression', random_state=41,
               reg_alpha=0.01, reg_lambda=0.01),
 LGBMRegressor(max_depth=30, metric=None, n_estimators=200, n_jobs=16,
               num_leaves=30, objective='regression', random_state=41,
               reg_alpha=0.01, reg_lambda=0.01),
 LGBMRegressor(max_depth=30, metric=None, n_estimators=200, n_jobs=16,
               num_leaves=30, objective='regression', random_state=41,
               reg_alpha=0.01, reg_lambda=0.01),
 LGBMRegressor(max_depth=30, metric=None, n_estimators=200, n_jobs=16,
               num_leaves=30, objective='regression', random_state=41,
               reg_alpha=0.01, reg_lambda=0.01)]

In [152]:
wmapes_train_2nd_deploy

[np.float64(0.25657381700617277),
 np.float64(0.2567791865661894),
 np.float64(0.27312535105692193),
 np.float64(0.23798824238282973),
 np.float64(0.13236879062020315)]

In [153]:
features_test.quantidade_0_1.value_counts()

quantidade_0_1
0.00      13755
1.00       3965
2.00       3479
3.00       2040
4.00       1417
          ...  
126.00        1
9.75          1
43.00         1
190.00        1
92.00         1
Name: count, Length: 117, dtype: int64

In [154]:
features_test.quantidade_1_4.value_counts()

quantidade_1_4
0.000000      10598
1.000000       5257
2.000000       4211
3.000000       2308
4.000000       1470
              ...  
1.958333          1
2.750000          1
168.000000        1
43.000000         1
81.000000         1
Name: count, Length: 117, dtype: int64

In [155]:
features_test.shape

(28909, 110)

In [156]:
X = features_test.drop(columns=target_cols + ['pdv_produto'])
outputs = []
for i in range(5):
    model = models_2nd_deploy[i]
    preds = np.round(model.predict(X))
    outputs.append(preds)

In [157]:
#data['pdv_produto'] = data['pdv'].astype(str) + '_' + data['produto'].astype(str)

In [158]:
pdv = features_test.pdv_produto.apply(lambda x: int(x.split('_')[0]))
produto = features_test.pdv_produto.apply(lambda x: int(x.split('_')[1]))

In [159]:
pdv.shape

(28909,)

In [160]:
pred_dfs = []
for i in range(5):
    pred_df = pd.DataFrame({
        'pdv': pdv,
        'produto': produto,
        'semana': i + 1,
        'quantidade': outputs[i]
    })
    pred_dfs.append(pred_df)

final_pred_df = pd.concat(pred_dfs, axis=0)

In [161]:
final_pred_df.reset_index(drop=True, inplace=True)
final_pred_df

,pdv,produto,semana,quantidade
0,1001371918471115422,1029370090212151375,1,3.0
1,1001371918471115422,2239307647969388381,1,7.0
2,1001371918471115422,3516587220187345473,1,2.0
3,1001371918471115422,4570988632387137438,1,4.0
4,1001371918471115422,6641579232767647477,1,11.0
...,...,...,...,...
144540,993703251780456435,9036639434778896273,5,2.0
144541,994716099648444815,8336391814233623319,5,4.0
144542,997907800111849739,2097372625156623809,5,-5.0
144543,997907800111849739,3516587220187345473,5,6.0


In [65]:
final_pred_df.quantidade.value_counts()

quantidade
 4.0      126554
 2.0       86853
 8.0       65104
 5.0       59531
 3.0       56649
           ...  
 245.0         1
 360.0         1
-44.0          1
-31.0          1
 424.0         1
Name: count, Length: 330, dtype: int64

In [171]:
final_pred_df.quantidade = final_pred_df.quantidade.astype(int)

In [172]:
final_pred_df[['semana', 'pdv', 'produto', 'quantidade']]

,semana,pdv,produto,quantidade
0,1,1001371918471115422,1029370090212151375,3
1,1,1001371918471115422,2239307647969388381,7
2,1,1001371918471115422,3516587220187345473,2
3,1,1001371918471115422,4570988632387137438,4
4,1,1001371918471115422,6641579232767647477,11
...,...,...,...,...
144540,5,993703251780456435,9036639434778896273,2
144541,5,994716099648444815,8336391814233623319,4
144542,5,997907800111849739,2097372625156623809,-5
144543,5,997907800111849739,3516587220187345473,6


In [173]:
final_pred_df.semana.value_counts(), final_pred_df.pdv.nunique(), final_pred_df.produto.nunique(), final_pred_df.shape

(semana
 1    28909
 2    28909
 3    28909
 4    28909
 5    28909
 Name: count, dtype: int64,
 3866,
 558,
 (144545, 4))

In [166]:
import autorootcwd #noqa 

deploy1 = pd.read_parquet("submissions/submission_model_v1.parquet")
final_pred_df.merge(deploy1, on=['pdv', 'produto', 'semana'], how='inner')

,pdv,produto,semana,quantidade_x,quantidade_y
0,1001371918471115422,1029370090212151375,1,3.0,4
1,1001371918471115422,2239307647969388381,1,7.0,9
2,1001371918471115422,3516587220187345473,1,2.0,2
3,1001371918471115422,4570988632387137438,1,4.0,4
4,1001371918471115422,6641579232767647477,1,11.0,12
...,...,...,...,...,...
144180,993703251780456435,9036639434778896273,5,2.0,15
144181,994716099648444815,8336391814233623319,5,4.0,18
144182,997907800111849739,2097372625156623809,5,-5.0,25
144183,997907800111849739,3516587220187345473,5,6.0,5


In [175]:
final_pred_df

,pdv,produto,semana,quantidade
0,1001371918471115422,1029370090212151375,1,3
1,1001371918471115422,2239307647969388381,1,7
2,1001371918471115422,3516587220187345473,1,2
3,1001371918471115422,4570988632387137438,1,4
4,1001371918471115422,6641579232767647477,1,11
...,...,...,...,...
144540,993703251780456435,9036639434778896273,5,2
144541,994716099648444815,8336391814233623319,5,4
144542,997907800111849739,2097372625156623809,5,-5
144543,997907800111849739,3516587220187345473,5,6


In [176]:
final_pred_df.to_parquet("submissions/submission_model_v2.parquet", index=False)

In [168]:
#final_pred_df.to_csv("submissions/submission_model_v1.csv", index=False, encoding='utf-8', sep=';')